In [52]:
"""
Author: Pavel Akapian
Email: pavel.akapian@gmail.com
Download and store tweets in MongoDB.
"""
from __future__ import print_function
import tweepy
import json
import datetime
from pymongo import MongoClient
 
class TweeMongoStreamListener(tweepy.StreamListener):
    """
    FILL IN COMMENTS
    """
    def __init__(self, end_time, collection_name,job_id,*args, **kwargs):
        self.endTime = end_time
        self.collectionName = collection_name
        super(TweeMongoStreamListener,self).__init__(*args,**kwargs)
     
    def on_connect(self):
        """Called when the connection is made"""
        print("You're connected to the streaming server.")
        
    def on_error(self, status_code):
        """This is called when an error occurs"""
        print('Error: ' + repr(status_code))
        return False
 
    def on_data(self, data):
        """This will be called each time we receive stream data"""
        client = MongoClient('localhost', 27017)
        #print("ON_DATA\n")
        db = client.cooldb
        datajson = json.loads(data)
        db[self.collectionName].insert_one(datajson)
        if datetime.datetime.utcnow() <=self.endTime:
            return True
        else:
            print("FINISHED COLLECTIONING")
            return False

In [53]:
def RunTwee(job_id,collection_name,track_list,minutes):
    CONSUMER_KEY = "OVW5fOSDPKMwpER1EcN57rRyB"
    CONSUMER_SECRET = "9UnSU3Ddj8Qqyjkws2L2LjD5vl2RoF7NvQsNSta6FvvL3lWbFU"
    ACCESS_TOKEN = "1965185312-jr5HFuWJ5YPulWCipvTYou8TcopKSoYWNxZuQA7"
    ACCESS_TOKEN_SECRET = "1prWACmbjcSNlOYMxkU31HX2debP0gER98Kw5Vqu49wfL"


    auth1 = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth1.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

    l = TweeMongoStreamListener(job_id = job_id,
                                collection_name = collection_name,
                                end_time=datetime.datetime.utcnow()+datetime.timedelta(seconds=60*minutes),
                                api = tweepy.API(wait_on_rate_limit=True))
    streamer = tweepy.Stream(auth=auth1, listener=l)
    streamer.filter(track=track_list)

In [54]:
from pytz import utc

from apscheduler.schedulers.background import BackgroundScheduler
from apscheduler.jobstores.mongodb import MongoDBJobStore
from apscheduler.executors.pool import ThreadPoolExecutor, ProcessPoolExecutor


jobstores = {
    'default': MongoDBJobStore()
}
executors = {
    'default': ThreadPoolExecutor(20),
    'processpool': ProcessPoolExecutor(5)
}
job_defaults = {
    'coalesce': False,
    'max_instances': 10
}
scheduler = BackgroundScheduler(jobstores=jobstores, executors=executors, job_defaults=job_defaults, timezone=utc)
scheduler.start()

In [56]:
scheduler.add_job(RunTwee,args=['ruby_job','CollRuby',["Ruby"],5],replace_existing=True,trigger='date',
                  run_date=datetime.datetime.utcnow()+datetime.timedelta(seconds=60),
                  timezone= utc, id="ruby_job")
scheduler.add_job(RunTwee,args=['python_job','CollPython',["Python"],10],replace_existing=True,trigger='date',
                  run_date=datetime.datetime.utcnow()+datetime.timedelta(seconds=10),
                  timezone= utc, id="python_job")

You're connected to the streaming server.
